In [ ]:
import json

import numpy as np
import torch
from torchvision import models

import shap

from torch.utils.data import random_split, DataLoader, Subset


In [ ]:
val_transform = transforms.Compose([
    transforms.Resize(540),         # 짧은 변을 540으로
    transforms.CenterCrop(512),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std =[0.229, 0.224, 0.225]
    )
])

data_root = '/home/work/workspace_ai/Artificlass/data_process/data/augmented_images_4'
full_dataset = datasets.ImageFolder(root=data_root, transform=None)
# train_ds = ImageFolder(root=data_root, transform=train_transform)
# val_ds   = ImageFolder(root=data_root, transform=val_transform)
style2idx   = full_dataset.class_to_idx.copy()
num_classes = len(style2idx)

In [ ]:
n = len(full_dataset)
indices = np.arange(n)
np.random.seed(seed)
np.random.shuffle(indices)

n_train = int(0.8 * n)
n_val   = int(0.1 * n)
train_idx = indices[:n_train]
val_idx   = indices[n_train:n_train+n_val]
test_idx  = indices[n_train+n_val:]

test_ds  = Subset(datasets.ImageFolder(root=data_root, transform=val_transform),
                  test_idx)

In [ ]:
loader_kwargs = dict(
    batch_size=16,
    num_workers=4,
    pin_memory=True,
    prefetch_factor=2,
    persistent_workers=True
)

test_loader  = DataLoader(test_ds,  shuffle=False, **loader_kwargs)


In [ ]:
class ResNet50Head(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        backbone = models.resnet50(pretrained=False)
        in_feats  = backbone.fc.in_features
        backbone.fc = nn.Identity()
        self.backbone = backbone

        self.drop = nn.Dropout(0.2)
        self.fc1  = nn.Linear(in_feats, 512)
        self.relu = nn.ReLU(inplace=True)
        self.fc2  = nn.Linear(512, 128)
        self.fc3  = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.backbone(x)        # → (batch, 2048)
        x = self.drop(x)
        x = self.relu(self.fc1(x))  # → (batch, 1024)
        x = self.drop(x)
        x = self.relu(self.fc2(x))
        x = self.drop(x)
        x = self.fc3(x)             # → (batch, num_classes)
        return x

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model  = ResNet50Head(num_classes).to(device)

In [ ]:
# Model load 

model_path='/home/work/workspace_ai/Artificlass/real_use_code/best_model_from_folder_pre.pth'
model.load_state_dict(torch.load(model_path))
model.eval()

In [ ]:
import os
import json
import numpy as np
import torch
from torchvision import models
from PIL import Image
import shap

# 1) 모델 불러오기
from backbone import ResNet50Head  # 실제 정의한 모듈 경로 맞춰주세요
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# data_root = '/home/work/workspace_ai/Artificlass/data_process/data/augmented_images_4'
# full_dataset = datasets.ImageFolder(root=data_root, transform=None)
# # train_ds = ImageFolder(root=data_root, transform=train_transform)
# # val_ds   = ImageFolder(root=data_root, transform=val_transform)
# style2idx   = full_dataset.class_to_idx.copy()
style2idx = {'Art Nouveau': 0, 'Baroque': 1, 'Expressionism': 2, 'Impressionism': 3, 'Post-Impressionism': 4, 'Realism': 5, 'Romantic': 6}
idx2style = {i:s for s,i in style2idx.items()}
num_classes = len(style2idx)
model = ResNet50Head(num_classes).to(device)
model.load_state_dict(torch.load('best_model_from_folder_pre.pth', map_location=device))
model.eval()

# 2) SHAP 입력용 normalize 함수
mean = np.array([0.485, 0.456, 0.406]).reshape((1,1,3))
std  = np.array([0.229, 0.224, 0.225]).reshape((1,1,3))
def preprocess(imgs):
    # imgs: numpy (B,H,W,3) in [0,1] or [0,255]
    if imgs.max() > 1.0:
        imgs = imgs / 255.0
    imgs = (imgs - mean) / std
    # B,H,W,3 → B,3,H,W
    return torch.from_numpy(imgs).permute(0,3,1,2).float().to(device)

# 3) 배경용(Background) 데이터 샘플
#    전체 테스트셋에서 랜덤하게 50장 정도 뽑아서 numpy array로 준비
from torchvision.datasets import ImageFolder
from torchvision import transforms

val_tf = transforms.Compose([
    transforms.Resize(540),
    transforms.CenterCrop(512),
    transforms.ToTensor()  # → [0,1], C×H×W
])

ds = ImageFolder(root='/home/work/workspace_ai/Artificlass/data_process/data/augmented_images_4',
                 transform=val_tf)
# 뒤에서 numpy로 바꾸려면 tensor→numpy
bg_indices = np.random.choice(len(ds), size=50, replace=False)
bg_imgs = []
for idx in bg_indices:
    img, _ = ds[idx]                # img: Tensor(3,512,512)
    bg_imgs.append(img.permute(1,2,0).numpy())  # H×W×C
bg_imgs = np.stack(bg_imgs, axis=0)  # (50,H,W,C)

# 4) 설명할 샘플 선택 (예: 2장)
to_explain = []
for idx in [10, 23]:
    img, _ = ds[idx]
    to_explain.append(img.permute(1,2,0).numpy())
to_explain = np.stack(to_explain, axis=0)  # (2,H,W,C)

# 5) GradientExplainer 생성
#    두 번째 인자는 ‘모델-출력-전’ 레이어를 지정할 수도 있지만,
#    전체 모델에 대해 입력 그라디언트 기반 설명을 원하면 (model, model.backbone) 처럼 전달
explainer = shap.GradientExplainer(
    (model, model.backbone),       # 또는 (model, model.backbone.layer4)
    preprocess(bg_imgs)            # background as Tensor(50,3,512,512)
)

# 6) SHAP 값 계산
#    ranked_outputs=1 → 가장 높은 클래스에 대한 explanation
shap_values, indexes = explainer.shap_values(
    preprocess(to_explain),       # Tensor(2,3,512,512)
    ranked_outputs=1,
    nsamples=100                   # 조절 가능
)

# 7) 클래스 이름 로드 (imagenet_class_index.json 필요)
# with open('imagenet_class_index.json') as f:
#     class_idx = json.load(f)
# # index → 이름 매핑
# idx_to_name = {int(k):v[1] for k,v in class_idx.items()}
# index_names = np.array([[ idx_to_name[i] for i in idx_list ]
#                         for idx_list in indexes])
index_names = np.array([
    [ idx2style[i] for i in idx_list ]
    for idx_list in indexes
])

# 8) 시각화
#    shap_values: list of arrays [ for each output-class: (2,3,512,512) ]
#    to_explain:      (2,512,512,3)
#    index_names:     (2,1)
shap_values = [ sv.transpose(0,2,3,1) for sv in shap_values ]
shap.image_plot(shap_values, to_explain, index_names)
